In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Import libraries

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [127]:

#Read csv
df = pd.read_csv('/content/drive/MyDrive/Jobathon/19 11 22/train_IxoE5JN.csv',index_col=[0],
                   parse_dates=['datetime'])
#,index_col=[1]
df1.head()

,energy,year,month,day,hour,saal
row_id,,,,,,
1,1259.985563,2008,3,1,0,1
2,1095.541500,2008,3,1,1,1
3,1056.247500,2008,3,1,2,1
4,1034.742000,2008,3,1,3,1
5,1026.334500,2008,3,1,4,1


In [ ]:
## Creating new month year date column from datetime values

In [106]:
date = df1.index.year
month = df1.index.month
day =df1.index.day
hours = df1.index.hour

print(date)

Int64Index([2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008,
            ...
            2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018],
           dtype='int64', name='datetime', length=94992)


In [107]:
df1['year'] = date
df1['month'] = month
df1['day'] = day
df1['hour'] = hours
saal = df1['year'] - 2007
df1['saal'] = saal
df1.head()

,row_id,energy,year,month,day,hour,saal
datetime,,,,,,,
2008-03-01 00:00:00,1,1259.985563,2008,3,1,0,1
2008-03-01 01:00:00,2,1095.541500,2008,3,1,1,1
2008-03-01 02:00:00,3,1056.247500,2008,3,1,2,1
2008-03-01 03:00:00,4,1034.742000,2008,3,1,3,1
2008-03-01 04:00:00,5,1026.334500,2008,3,1,4,1


## Checking is their null values

In [108]:
df1.isnull().sum()

row_id       0
energy    1900
year         0
month        0
day          0
hour         0
saal         0
dtype: int64

In [109]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 94992 entries, 2008-03-01 00:00:00 to 2018-12-31 23:00:00
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   row_id  94992 non-null  int64  
 1   energy  93092 non-null  float64
 2   year    94992 non-null  int64  
 3   month   94992 non-null  int64  
 4   day     94992 non-null  int64  
 5   hour    94992 non-null  int64  
 6   saal    94992 non-null  int64  
dtypes: float64(1), int64(6)
memory usage: 5.8 MB


In [110]:
df1['energy'] = df1['energy'].fillna(df1['energy'].mean())

#or we can drop it
df1 = df1.dropna(axis=0)

In [111]:
df1 = df1.set_index('row_id')
df1.head()

,energy,year,month,day,hour,saal
row_id,,,,,,
1,1259.985563,2008,3,1,0,1
2,1095.541500,2008,3,1,1,1
3,1056.247500,2008,3,1,2,1
4,1034.742000,2008,3,1,3,1
5,1026.334500,2008,3,1,4,1


## Visualizing Data

In [ ]:
# DISCLAIMER: 'df' refers to the data you passed in when calling 'dtale.show'

import pandas as pd

if isinstance(df, (pd.DatetimeIndex, pd.MultiIndex)):
	df = df.to_frame(index=False)

# remove any pre-existing indices for ease of use in the D-Tale code, but this is not required
df = df.reset_index().drop('index', axis=1, errors='ignore')
df.columns = [str(c) for c in df.columns]  # update columns to strings in case they are numbers

chart_data = pd.concat([
	df['datetime'],
	df['energy'],
	pd.Series(df.set_index('datetime').index.to_period('Y').to_timestamp(how='end').values, index=df.index, name='datetime|Y'),
], axis=1)
chart_data = chart_data.sort_values(['datetime|Y', 'datetime'])
chart_data = chart_data.rename(columns={'datetime': 'x'})
chart_data = chart_data.dropna()
# WARNING: This is not taking into account grouping of any kind, please apply filter associated with
#          the group in question in order to replicate chart. For this we're using '"""`datetime`.dt.year == 2008"""'
chart_data = chart_data.query("""`datetime`.dt.year == 2008""")

import plotly.graph_objs as go

charts = []
charts.append(go.Bar(
	x=chart_data['x'],
	y=chart_data['energy'],
	name='(datetime.dt.year: 2008)'
))
figure = go.Figure(data=charts, layout=go.Layout({
    'barmode': 'group',
    'legend': {'orientation': 'h'},
    'title': {'text': '(datetime.dt.year: 2008) - energy by datetime'},
    'xaxis': {'title': {'text': 'datetime'}},
    'yaxis': {'title': {'text': 'energy'}, 'type': 'linear'}
}))

# If you're having trouble viewing your chart in your notebook try passing your 'chart' into this snippet:
#
# from plotly.offline import iplot, init_notebook_mode
#
# init_notebook_mode(connected=True)
# for chart in charts:
#     chart.pop('id', None) # for some reason iplot does not like 'id'
# iplot(figure)

## Train - Test split 

In [112]:
x = pd.DataFrame()
y = pd.DataFrame()
x = df1.drop(['energy','year'],axis=1)
y = df1['energy']

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.20,random_state = 13)

In [113]:
print(x.shape)
print(y.shape)

(94992, 4)
(94992,)


## Training a simple Linear Regression and Svr to check how much it can be helpful

In [114]:

reg = LinearRegression()
model = reg.fit(x,y)

In [115]:
model.score(x,y)

0.469092079572745

In [116]:

lm = LinearRegression()
model = lm.fit(x_train,y_train)
from sklearn.metrics import mean_squared_error
y_pred = model.predict(x_test)
np.sqrt(mean_squared_error(y_test,y_pred))

266.9531584723053

In [117]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

import numpy as np
regr =  SVR(C=1.0, epsilon=0.2)
regr.fit(x_train,y_train)

from sklearn.metrics import mean_squared_error
y_pred = model.predict(x_test)
np.sqrt(mean_squared_error(y_test,y_pred))

266.9531584723053

In [94]:
from sklearn.metrics import mean_squared_error
y_pred = regr.predict(x_test)
np.sqrt(mean_squared_error(y_test,y_pred))

260.2859414411414

So mean absolute error by LR and SVR are 266 and 260

In [ ]:

fig = plt.line(df, x='datetime', y='energy')
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [128]:
df['energy'] = df['energy'].fillna(df['energy'].mean())

## So now we are trying fbprophet of facebook for univariate demand forecasting.

In [129]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot
df.rename(columns={'datetime': 'ds', 'energy': 'y'}, inplace=True)
train_data = df.sample(frac=0.8, random_state=10)

Converting column name to ds and y as required.

In [130]:
df.head()

,ds,y
row_id,,
1,2008-03-01 00:00:00,1259.985563
2,2008-03-01 01:00:00,1095.541500
3,2008-03-01 02:00:00,1056.247500
4,2008-03-01 03:00:00,1034.742000
5,2008-03-01 04:00:00,1026.334500


Separating out train and test data

In [131]:
validation_data = df.drop(train_data.index)

print(f'training data size : {train_data.shape}')
print(f'validation data size : {validation_data.shape}')

train_data = train_data.reset_index()
validation_data = validation_data.reset_index()

training data size : (75994, 2)
validation data size : (18998, 2)


In [52]:
!pip install pystan~=2.14
!pip install fbprophet
from sklearn.metrics import mean_absolute_error
from fbprophet import Prophet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 67.3 MB 158 kB/s 
  Attempting uninstall: pystan
    Found existing installation: pystan 3.3.0
    Uninstalling pystan-3.3.0:
      Successfully uninstalled pystan-3.3.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached fbprophet-0.7.1.tar.gz (64 kB)
  Created wheel for fbprophet: filename=fbprophet-0.7.1-py3-none-any.whl size=6638895 sha256=e9a503f60bd398eb24ec33a22383855ff42958cdf7349d6c3f84bd65f67db673
  Stored in directory: /root/.cache/pip/wheels/cd/a1/12/db63ff624de492fe6cccf676091a0860fdde2ffde4bc3280e2
Successfully built fbprophet


Calling model and perfrming fit operation.
1. As energy demand is effected by months, just like in summer or particular month demand of energy hiked.
2. Also adding country holidays, because that time energy demand increases.

In [132]:
model = Prophet(weekly_seasonality=True,yearly_seasonality=True)
model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
model.add_country_holidays(country_name='UK')
model.fit(df)

To check model performance evaluating on test data.

In [133]:
prediction = model.predict(pd.DataFrame({'ds':validation_data['ds']}))
y_actual = validation_data['y']
y_predicted = prediction['yhat']
y_predicted = y_predicted.astype(int)
mean_absolute_error(y_actual, y_predicted)

134.3363917915947

Plotting resuts.

In [ ]:
from plotly.subplots import make_subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=validation_data['ds'], y=y_actual, name="actual targets"),
    secondary_y=False,)
fig.add_trace(
    go.Scatter(x=validation_data['ds'], y=y_predicted, name="predicted targets"),
    secondary_y=True,)
fig.update_layout(
    title_text="Actual vs Predicted Targets")
fig.update_xaxes(title_text="Timeline")
fig.update_yaxes(title_text="actual targets", secondary_y=False)
fig.update_yaxes(title_text="predicted targets", secondary_y=True)
fig.show()

## New Generating Result

In [134]:
test = pd.read_csv('/content/drive/MyDrive/Jobathon/19 11 22/test_WudNWDM.csv',
                   parse_dates=['datetime'])

test.head()

,row_id,datetime
0,94993,2019-01-01 00:00:00
1,94994,2019-01-01 01:00:00
2,94995,2019-01-01 02:00:00
3,94996,2019-01-01 03:00:00
4,94997,2019-01-01 04:00:00


Predicting Result

In [135]:
prediction = model.predict(pd.DataFrame({'ds':test['datetime']}))
y_predicted = prediction['yhat']
y_predicted = y_predicted.astype(int)
print(y_predicted)

0        2035
1        1960
2        1906
3        1878
4        1881
         ... 
26299    2531
26300    2543
26301    2527
26302    2476
26303    2395
Name: yhat, Length: 26304, dtype: int64


In [136]:
test['energy'] = y_predicted
test.head()

,row_id,datetime,energy
0,94993,2019-01-01 00:00:00,2035
1,94994,2019-01-01 01:00:00,1960
2,94995,2019-01-01 02:00:00,1906
3,94996,2019-01-01 03:00:00,1878
4,94997,2019-01-01 04:00:00,1881


In [137]:
test.head()

,row_id,datetime,energy
0,94993,2019-01-01 00:00:00,2035
1,94994,2019-01-01 01:00:00,1960
2,94995,2019-01-01 02:00:00,1906
3,94996,2019-01-01 03:00:00,1878
4,94997,2019-01-01 04:00:00,1881


In [138]:
result = pd.DataFrame()
result = test.drop(['datetime'],axis=1)
result.to_csv("result1.csv",index=False)